Dataset Creation

In [0]:
import os


# find the dataset here: https://www.unb.ca/cic/datasets/invesandmal2019.html
!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/Adware.zip
!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/PremiumSMS.zip
!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/Ransomware.zip
!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/SMS.zip
!wget http://205.174.165.80/CICDataset/CICInvesAndMal2019/Dataset/APKs/Scareware.zip

#!wget http:........Benign_2015.zip
#!wget http:........Benign_2016.zip
#!wget http:........Benign_2017.zip

!unzip /content/Adware.zip
os.remove('/content/Adware.zip')
!unzip /content/PremiumSMS.zip
os.remove('/content/PremiumSMS.zip')
!unzip /content/SMS.zip
os.remove('/content/SMS.zip')
!unzip /content/Ransomware.zip
os.remove('/content/Ransomware.zip')
!unzip /content/Scareware.zip
os.remove('/content/Scareware.zip')

!unzip /content/Benign_2015.zip
os.remove('/content/Benign_2015.zip')
!unzip /content/Benign_2016.zip
os.remove('/content/Benign_2016.zip')
!unzip /content/Benign_2017.zip
os.remove('/content/Benign_2017.zip')

In [0]:
import shutil

#!git clone https://github.com/ethicalhackeragnidhra/Android-Malwares.git /content/mal
!git clone https://github.com/RyzenElstra/Android-Malwares.git /content/mal
#!git clone https://github.com/ashishb/android-malware.git /content/mal

In [0]:
#import shutil
#shutil.rmtree('/content/mal')

In [0]:

#!unzip -P 'infected' -r /content/mal

In [0]:
import zipfile

# pswd = 'infected'

for root, dirs, files in os.walk('/content/mal'):
  for file in files:
    if file.endswith('.zip'):
      try:
        with zipfile.ZipFile(os.path.join(root,file), 'r') as zip_ref:
          #zip_ref.setpassword(pwd = bytes(pswd, 'utf-8'))
          zip_ref.extractall()
        os.remove(os.path.join(root,file))
      except:
        os.remove(os.path.join(root,file))
for root, dirs, files in os.walk('/content/mal'):
  for file in files:
    if file.endswith('.apk'):
      print(file)
      shutil.copy(os.path.join(root,file), '/content/dataset/malign/')
      os.remove(os.path.join(root,file))


In [0]:

import shutil

msources = ['/content/Adware', '/content/PremiumSMS', '/content/SMS', '/content/Ransomware', '/content/Scareware']
bsources = ['/content/Benign_2015', '/content/Benign_2016', '/content/Benign_2017']
for s in msources:
  for root, dirs, files in os.walk(s):
    for file in files:
      if file.endswith('.apk'):
        print(file)
        shutil.copy(os.path.join(root,file), '/content/dataset/malign/')
for p in bsources:
  for root, dirs, files in os.walk(p):
    for file in files:
      if file.endswith('.apk'):
        print(file)
        shutil.copy(os.path.join(root,file), '/content/dataset/benign/')
        os.remove(os.path.join(root,file))

In [0]:
!pip install androguard

In [0]:
import os, sys, stat 
malign = '/content/dataset/malign'
benign = '/content/dataset/benign'
countM = 0
countB = 0

for entry in os.scandir(benign):
  if entry.path.endswith(".apk") and entry.is_file():
          countB += 1
          #if countB> 550:
          # os.remove(entry.path)
for entry in os.scandir(malign):
  if entry.path.endswith(".apk") and entry.is_file():
          countM += 1
          
print(countB, countM)


1150 529


In [0]:
import os, sys, stat 
from androguard.core.bytecodes.apk import APK

malign = '/content/dataset/malign'
benign = '/content/dataset/benign'

permissions =  []
countM = 0
countB = 0

for entry in os.scandir(benign):
  if entry.path.endswith(".apk") and entry.is_file():
          try:
            a = APK(entry.path)
            perm = a.get_permissions()
            countB += 1
            for per in perm:
              if per not in permissions and per.startswith('android.permission'):
                permissions.append(per)
          except:
            os.remove(entry.path)

for entry in os.scandir(malign):
  if entry.path.endswith(".apk") and entry.is_file():
          try:
            a = APK(entry.path)
            perm = a.get_permissions()
            countM += 1
            for per in perm:
              if per not in permissions and per.startswith('android.permission'):
                permissions.append(per)
          except:
            os.remove(entry.path)       


In [0]:
print(countB,countM)

1150 529


In [0]:
import pandas as pd
default = open('/content/drive/My Drive/Android-Malware-Detection/DefaultPermList.txt','r').readlines()
perms = [s.rstrip('\n') for s in default]
for p in permissions:
  if p not in perms:
    perms.append(p)

In [0]:
print(len(perms))
with open('/content/drive/My Drive/Android-Malware-Detection/permissions.txt','w+') as f:
  for p in perms:
    f.write(p + '\n')

428


In [0]:
pe = []
with open('/content/drive/My Drive/Android-Malware-Detection/permissions.txt','r') as a:
  content = a.readlines()
  for line in content:
    cur_perm = line[:-1]
    pe.append(cur_perm)
len(pe)



428

In [0]:
dataset_df = pd.DataFrame(columns=perms)
dataset_b = {}
id = 0
for entry in os.scandir(benign):
  if entry.path.endswith(".apk") and entry.is_file():
    a = APK(entry.path)
    perm = a.get_permissions()
    id += 1
    for d in perms:
      if d in perm:
        dataset_b[d]=1
      else:
        dataset_b[d]=0
    dataset_b['class'] = 'benign'
    dataset_df=dataset_df.append(dataset_b, ignore_index=True)
dataset_m = {}
for entry in os.scandir(malign):
  if entry.path.endswith(".apk") and entry.is_file():
    a = APK(entry.path)
    perm = a.get_permissions()
    id += 1
    for d in perms:
      if d in perm:
        dataset_m[d]=1
      else:
        dataset_m[d]=0
    dataset_m['class'] = 'malign'
    dataset_df=dataset_df.append(dataset_m, ignore_index=True)

In [0]:
dataset_df.to_csv('/content/drive/My Drive/Android-Malware-Detection/android_dataset.csv', index=False)

# **SVM : SVC with Grid Search to tune model**

In [0]:
import sklearn
import pickle
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix 
Y = dataset_df['class']
X = dataset_df.drop(['class'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [0]:
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
			'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
			'kernel': ['rbf']} 

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 

# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.691, total=   0.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.684, total=   0.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.7s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.688, total=   0.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.687, total=   0.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.687, total=   0.8s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.814, total=   0.7s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.874, total=   0.7s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.877, total=   0.6s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.825, total=   0.6s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] .

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:  1.2min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [0]:
print(grid.best_params_) 
grid_predictions = grid.predict(X_test) 

# print classification report 
print(classification_report(y_test, grid_predictions)) 
pickle.dump(grid, open('/content/drive/My Drive/Android-Malware-Detection/svc_new.pkl', 'wb'))

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
              precision    recall  f1-score   support

      benign       0.91      0.93      0.92       230
      malign       0.85      0.80      0.83       106

    accuracy                           0.89       336
   macro avg       0.88      0.87      0.87       336
weighted avg       0.89      0.89      0.89       336



Predict

In [0]:
from androguard.core.bytecodes.apk import APK

def predict(apk):
  vector = {}
  a = APK(apk)
  perm = a.get_permissions()
  print(perm)
  for d in perms:
    if d in perm:
      vector[d]=1
    else:
      vector[d]=0
  input = [ v for v in vector.values() ]
  print(input)
  print(grid.predict([input]))

predict('/content/Ransomware/PornDroid/1c53e2c34d1219a2fae8fcf8ec872ac8.apk')
predict('/content/dataset/benign/a.envisionmobile.caa.apk')

['android.permission.RECEIVE_BOOT_COMPLETED', 'com.android.browser.permission.READ_HISTORY_BOOKMARKS', 'android.permission.WRITE_SETTINGS', 'android.permission.GET_ACCOUNTS', 'android.permission.READ_PROFILE', 'android.permission.GET_TASKS', 'android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.WAKE_LOCK', 'android.permission.INTERNET', 'android.permission.VIBRATE', 'android.permission.READ_PHONE_STATE', 'android.permission.READ_CONTACTS', 'android.permission.SYSTEM_ALERT_WINDOW', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.CAMERA', 'android.permission.ACCESS_NETWORK_STATE']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# **Multilayer Perceptron (Simple Artificial Neural Network)**

In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix 

import numpy as np
from sklearn.preprocessing import LabelEncoder

In [0]:
Y = dataset_df['class']
X = dataset_df.drop(['class'], axis=1)
encoder = LabelEncoder().fit(Y)
Y = encoder.transform(Y)
#print(encoder.transform(['malign']))
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [0]:
AN = Sequential()
AN.add(Dense(256, activation='relu', input_dim=428))
AN.add(Dropout(0.2))
AN.add(Dense(128, activation='relu'))
AN.add(Dropout(0.2))
AN.add(Dense(128, activation='relu'))
AN.add(Dropout(0.2))
AN.add(Dense(32, activation='relu'))
AN.add(Dropout(0.2))
AN.add(Dense(1, activation='sigmoid'))
AN.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

AN.fit(X_train, y_train, epochs=100, batch_size=32)


scores = AN.evaluate(X_test, y_test)
for i in range(len(scores)):
  print("\n%s: %.2f%%" % (AN.metrics_names[i], scores[i]*100))


Epoch 1/100
1343/1343 [==============================] - 0s 289us/step - loss: 0.6613 - accuracy: 0.6783
Epoch 2/100
1343/1343 [==============================] - 0s 134us/step - loss: 0.6325 - accuracy: 0.6910
Epoch 3/100
1343/1343 [==============================] - 0s 146us/step - loss: 0.6156 - accuracy: 0.6895
Epoch 4/100
1343/1343 [==============================] - 0s 140us/step - loss: 0.5983 - accuracy: 0.6895
Epoch 5/100
1343/1343 [==============================] - 0s 145us/step - loss: 0.5820 - accuracy: 0.6910
Epoch 6/100
1343/1343 [==============================] - 0s 147us/step - loss: 0.5618 - accuracy: 0.7029
Epoch 7/100
1343/1343 [==============================] - 0s 144us/step - loss: 0.5505 - accuracy: 0.7148
Epoch 8/100
1343/1343 [==============================] - 0s 145us/step - loss: 0.5281 - accuracy: 0.7319
Epoch 9/100
1343/1343 [==============================] - 0s 143us/step - loss: 0.5034 - accuracy: 0.7513
Epoch 10/100
1343/1343 [==============================]

In [0]:
from androguard.core.bytecodes.apk import APK

def predict(apk):
  vector = {}
  a = APK(apk)
  perm = a.get_permissions()
  print(perm)
  for d in perms:
    if d in perm:
      vector[d]=1
    else:
      vector[d]=0
  input = [ v for v in vector.values() ]
  print(input)
  print(AN.predict([[input]]))

predict('/content/Ransomware/PornDroid/1c53e2c34d1219a2fae8fcf8ec872ac8.apk')
predict('/content/drive/My Drive/Android-Malware-Detection/dataset/benign/com.whatsapp.apk')

['android.permission.RECEIVE_BOOT_COMPLETED', 'com.android.browser.permission.READ_HISTORY_BOOKMARKS', 'android.permission.WRITE_SETTINGS', 'android.permission.GET_ACCOUNTS', 'android.permission.READ_PROFILE', 'android.permission.GET_TASKS', 'android.permission.WRITE_EXTERNAL_STORAGE', 'android.permission.WAKE_LOCK', 'android.permission.INTERNET', 'android.permission.VIBRATE', 'android.permission.READ_PHONE_STATE', 'android.permission.READ_CONTACTS', 'android.permission.SYSTEM_ALERT_WINDOW', 'android.permission.READ_EXTERNAL_STORAGE', 'android.permission.CAMERA', 'android.permission.ACCESS_NETWORK_STATE']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Requested API level 29 is larger than maximum we have, returning API level 28 instead.


['android.permission.READ_SYNC_STATS', 'android.permission.RECEIVE_BOOT_COMPLETED', 'android.permission.WRITE_EXTERNAL_STORAGE', 'com.huawei.android.launcher.permission.READ_SETTINGS', 'android.permission.MODIFY_AUDIO_SETTINGS', 'com.android.launcher.permission.INSTALL_SHORTCUT', 'android.permission.RECEIVE_SMS', 'com.huawei.android.launcher.permission.CHANGE_BADGE', 'com.htc.launcher.permission.READ_SETTINGS', 'com.whatsapp.permission.REGISTRATION', 'android.permission.READ_CONTACTS', 'android.permission.CHANGE_WIFI_STATE', 'android.permission.NFC', 'android.permission.ACCESS_COARSE_LOCATION', 'android.permission.BLUETOOTH', 'com.android.launcher.permission.UNINSTALL_SHORTCUT', 'android.permission.ACCESS_FINE_LOCATION', 'android.permission.ACCESS_WIFI_STATE', 'android.permission.ACCESS_NETWORK_STATE', 'com.huawei.android.launcher.permission.WRITE_SETTINGS', 'android.permission.MANAGE_OWN_CALLS', 'com.whatsapp.permission.BROADCAST', 'android.permission.GET_TASKS', 'android.permission.B

In [0]:
pickle.dump(AN, open('/content/drive/My Drive/Android-Malware-Detection/ANN2.pkl', 'wb'))